In [18]:
import pandas as pd
import numpy as np

In [29]:
dataset = pd.read_csv("cafe_sales.csv")
dataset

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2,4,Credit Card,Takeaway,9/8/2023
1,TXN_4977031,Cake,4,3,12,Cash,In-store,5/16/2023
2,TXN_4271903,Cookie,4,1,ERROR,Credit Card,In-store,7/19/2023
3,TXN_7034554,Salad,2,5,10,UNKNOWN,UNKNOWN,4/27/2023
4,TXN_3160411,Coffee,2,2,4,Digital Wallet,In-store,6/11/2023
...,...,...,...,...,...,...,...,...
9995,TXN_7672686,Coffee,2,2,4,NaN,UNKNOWN,8/30/2023
9996,TXN_9659401,NaN,3,NaN,3,Digital Wallet,NaN,6/2/2023
9997,TXN_5255387,Coffee,4,2,8,Digital Wallet,NaN,3/2/2023
9998,TXN_7695629,Cookie,3,NaN,3,Digital Wallet,NaN,12/2/2023


In [16]:
dataset.dtypes

Transaction ID      object
Item                object
Quantity            object
Price Per Unit      object
Total Spent         object
Payment Method      object
Location            object
Transaction Date    object
dtype: object

In [32]:
# Replace "ERROR" and "UNKNOWN" with NaN
dataset.replace("ERROR", np.nan, inplace=True)
dataset.replace("UNKNOWN", np.nan, inplace=True)

In [41]:
# Convert to numeric types
columns_to_fix = ["Quantity", "Price Per Unit", "Total Spent"]
for columnName in columns_to_fix:
    dataset[columnName] = pd.to_numeric(dataset[columnName], errors='coerce')

In [42]:
dataset

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2.0,2.0,2.0,Credit Card,Takeaway,9/8/2023
1,TXN_4977031,Cake,4.0,3.0,3.0,Cash,In-store,5/16/2023
2,TXN_4271903,Cookie,4.0,1.0,1.0,Credit Card,In-store,7/19/2023
3,TXN_7034554,Salad,2.0,5.0,5.0,NaN,NaN,4/27/2023
4,TXN_3160411,Coffee,2.0,2.0,2.0,Digital Wallet,In-store,6/11/2023
...,...,...,...,...,...,...,...,...
9995,TXN_7672686,Coffee,2.0,2.0,2.0,NaN,NaN,8/30/2023
9996,TXN_9659401,NaN,3.0,NaN,NaN,Digital Wallet,NaN,6/2/2023
9997,TXN_5255387,Coffee,4.0,2.0,2.0,Digital Wallet,NaN,3/2/2023
9998,TXN_7695629,Cookie,3.0,NaN,NaN,Digital Wallet,NaN,12/2/2023


In [43]:
dataset.dtypes

Transaction ID       object
Item                 object
Quantity            float64
Price Per Unit      float64
Total Spent         float64
Payment Method       object
Location             object
Transaction Date     object
dtype: object

In [44]:
dataset.isnull().sum()

Transaction ID         0
Item                 969
Quantity             479
Price Per Unit       533
Total Spent          533
Payment Method      3178
Location            3961
Transaction Date     460
dtype: int64

In [45]:
from Univariate import Univariate
quan,qual = Univariate.quanQual(dataset)

In [46]:
quan

['Quantity', 'Price Per Unit', 'Total Spent']

In [47]:
qual

['Transaction ID', 'Item', 'Payment Method', 'Location', 'Transaction Date']

In [50]:
#impute Numerical Columns
from sklearn.impute import SimpleImputer
from scipy.stats import shapiro

for columnName in quan:
    if dataset[columnName].isnull().sum() > 0:
        skew_val = dataset[columnName].skew()
        col_data = dataset[columnName].dropna() #removes any NaN (missing) values in the specified column - would disrupt the calculation.

        # Default to median
        strategy = 'median'

        #Strategy Decision: 
        #Check normality with Shapiro-Wilk Test (only if enough data)
        if abs(skew_val) < 0.5:
                strategy = 'mean'
        elif abs(skew_val) < 1 and len(col_data) > 3:
            stat, p = shapiro(col_data)
            if p > 0.05:
                strategy = 'mean'


        print(f"{columnName}: Skew = {skew_val:.2f}, {strategy}")

        #Apply imputation
        imp = SimpleImputer(missing_values=np.nan, strategy=strategy)
        imp.fit(dataset[[columnName]])
        dataset[[columnName]] = imp.transform(dataset[[columnName]])           

Quantity: Skew = -0.01, mean
Price Per Unit: Skew = 0.00, mean
Total Spent: Skew = 0.00, mean


In [51]:
df = pd.DataFrame(dataset, columns = quan) 
df

,Quantity,Price Per Unit,Total Spent
0,2.0,2.000000,2.000000
1,4.0,3.000000,3.000000
2,4.0,1.000000,1.000000
3,2.0,5.000000,5.000000
4,2.0,2.000000,2.000000
...,...,...,...
9995,2.0,2.000000,2.000000
9996,3.0,2.949984,2.949984
9997,4.0,2.000000,2.000000
9998,3.0,2.949984,2.949984


In [52]:
#impute Categorical Columns
for columnName in qual:
    if dataset[columnName].isnull().sum() > 0:
        imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
        imp.fit(dataset[[columnName]])
        dataset[[columnName]] = imp.transform(dataset[[columnName]])   

In [53]:
df = pd.DataFrame(dataset, columns = qual) 
df

,Transaction ID,Item,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,Credit Card,Takeaway,9/8/2023
1,TXN_4977031,Cake,Cash,In-store,5/16/2023
2,TXN_4271903,Cookie,Credit Card,In-store,7/19/2023
3,TXN_7034554,Salad,Digital Wallet,Takeaway,4/27/2023
4,TXN_3160411,Coffee,Digital Wallet,In-store,6/11/2023
...,...,...,...,...,...
9995,TXN_7672686,Coffee,Digital Wallet,Takeaway,8/30/2023
9996,TXN_9659401,Juice,Digital Wallet,Takeaway,6/2/2023
9997,TXN_5255387,Coffee,Digital Wallet,Takeaway,3/2/2023
9998,TXN_7695629,Cookie,Digital Wallet,Takeaway,12/2/2023
